<a href="https://colab.research.google.com/github/ethanmjansen/DS-Unit-2-Applied-Modeling/blob/master/module1/LS_DS10_assignment_applied_modeling_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 3, Module 1*

---


# Define ML problems

You will use your portfolio project dataset for all assignments this sprint.

## Assignment

Complete these tasks for your project, and document your decisions.

- [ ] Choose your target. Which column in your tabular dataset will you predict?
- [ ] Is your problem regression or classification?
- [ ] How is your target distributed?
    - Classification: How many classes? Are the classes imbalanced?
    - Regression: Is the target right-skewed? If so, you may want to log transform the target.
- [ ] Choose your evaluation metric(s).
    - Classification: Is your majority class frequency >= 50% and < 70% ? If so, you can just use accuracy if you want. Outside that range, accuracy could be misleading. What evaluation metric will you choose, in addition to or instead of accuracy?
    - Regression: Will you use mean absolute error, root mean squared error, R^2, or other regression metrics?
- [ ] Choose which observations you will use to train, validate, and test your model.
    - Are some observations outliers? Will you exclude them?
    - Will you do a random split or a time-based split?
- [ ] Begin to clean and explore your data.
- [ ] Begin to choose which features, if any, to exclude. Would some features "leak" future information?

If you haven't found a dataset yet, do that today. [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2) and choose your dataset.

#Pre Work Stuff


In [0]:
import pandas as pd

In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
df = pd.read_csv('2018_Central_Park_Squirrel_Census_-_Squirrel_Data (1).csv')

#Choose a Target

In [9]:
df.head(2)

,X,Y,Unique Squirrel ID,Hectare,Shift,Date,Hectare Squirrel Number,Age,Primary Fur Color,Highlight Fur Color,Combination of Primary and Highlight Color,Color notes,Location,Above Ground Sighter Measurement,Specific Location,Running,Chasing,Climbing,Eating,Foraging,Other Activities,Kuks,Quaas,Moans,Tail flags,Tail twitches,Approaches,Indifferent,Runs from,Other Interactions,Lat/Long,Zip Codes,Community Districts,Borough Boundaries,City Council Districts,Police Precincts
0,-73.956134,40.794082,37F-PM-1014-03,37F,PM,10142018,3,NaN,NaN,NaN,+,NaN,NaN,NaN,NaN,False,False,False,False,False,NaN,False,False,False,False,False,False,False,False,NaN,POINT (-73.9561344937861 40.7940823884086),NaN,19,4,19,13
1,-73.957044,40.794851,37E-PM-1006-03,37E,PM,10062018,3,Adult,Gray,Cinnamon,Gray+Cinnamon,NaN,Ground Plane,FALSE,NaN,True,False,False,False,False,NaN,False,False,False,False,False,False,False,True,me,POINT (-73.9570437717691 40.794850940803904),NaN,19,4,19,13


In [10]:
#Looking for weird instances where a squirrel was indifferent but ran away
df['run_ind'] = df['Runs from'] & df['Indifferent'] == True
df['run_ind'].value_counts()

False    2991
True       32
Name: run_ind, dtype: int64

In [11]:
#Dropping those instances
indexNames = df[ (df['Runs from'] == True) & (df['Indifferent'] == True) ].index
df.drop(indexNames , inplace=True)
df['run_ind'].value_counts()

False    2991
Name: run_ind, dtype: int64

In [12]:
#Looking for weird instances where a squirrel was all three
df['all_false'] = df['Runs from'] & df['Indifferent'] & df['Approaches'] == False
df['all_false'].value_counts()

True    2991
Name: all_false, dtype: int64

In [13]:
#Looking for weird instances where a squirrl was all three
df['all_true'] = df['Runs from'] & df['Indifferent'] & df['Approaches'] == True
df['all_true'].value_counts()

False    2991
Name: all_true, dtype: int64

In [14]:
#This is my target
df['Target'] = df['Approaches'] | df['Indifferent'] == True
df['Target'].value_counts()

True     1581
False    1410
Name: Target, dtype: int64

#Regression or Classification?




This is a binary classification problem, because squirrels will either, run away when they see a person, or they will approach/not care when around a person. 

# How is Target Distributed?

In [0]:
#Defining y 
y = df['Target']

In [16]:
#Number of unique instances in y 
y.nunique()

2

In [17]:
#Majority class 
y.value_counts(normalize=True).max()

0.5285857572718154

Based on these value counts, it seems that the target is evenly distributed between running and not running away. 

#Evaluation Metrics


In [18]:
y.value_counts(normalize=True)

True     0.528586
False    0.471414
Name: Target, dtype: float64

My Majority Class is 52% of the data and since that is above 50% and less than 70% I would say that Accuray is a pretty good metric to use in how effective my model is. I would also add that precision would be important becasue if you approach a squirrel and it reacts violently someone could get hurt or you could cause distress to the squirrel. 

#Which Observations to Train/Validate/Test








*   There are no outliers in this data, being a classification problem there aren't any classes that are out of the range of what I would consider normal.  

*   I will do a Random Train/Val/Test split because the observations are only during the year of 2018 in the month of October. There is no reason to split based on date. 

#Clean and Explore Data

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
#Dropping Columns I was using to inspect for odd instances 
#wile making my target column 
columns_to_drop = ['run_ind', 'all_false', 'all_true']
df = df.drop(columns=columns_to_drop)

In [21]:
#Looking at null values and seeing how this would effect my data
df.isnull().sum()

X                                                0
Y                                                0
Unique Squirrel ID                               0
Hectare                                          0
Shift                                            0
Date                                             0
Hectare Squirrel Number                          0
Age                                            121
Primary Fur Color                               55
Highlight Fur Color                           1077
Combination of Primary and Highlight Color       0
Color notes                                   2814
Location                                        62
Above Ground Sighter Measurement               112
Specific Location                             2522
Running                                          0
Chasing                                          0
Climbing                                         0
Eating                                           0
Foraging                       

In [22]:
#Value counts of activities
df['Other Activities'].value_counts()

digging                                      19
sitting                                      16
walking                                      14
burying                                      12
playing                                      11
                                             ..
digging deeeep                                1
digging,chewing on tree—has nut in mouth!     1
eating (osage orange)                         1
standing on back legs                         1
in hollow                                     1
Name: Other Activities, Length: 302, dtype: int64

In [23]:
#How many kinds of other Activities
df['Other Activities'].nunique()

302

In [24]:
#Looking at the kinds of Activities
df['Other Activities'].unique()

array([nan, 'unknown', 'wrestling with mother', 'grooming', 'walking',
       'moving slowly', 'sitting',
       'eating (ate upside down on a tree — #jealous)',
       'running (with nut)', 'playing with #5', 'hiding nut',
       'drank from a pond of rain water', 'chasing (#4 across trees)',
       'gathering acorns',
       'eating (a mushroom),circles around us,really fat,scratching himself,grooming activity',
       'digging', 'walking on branch',
       'being chased,was pushed by other squirrel', 'climbing (tree)',
       'chasing #8', 'still', 'stop to look at me—then ran',
       'chasing (a chipmunk)', 'took nut to bury', 'flying',
       'stole (found?) an entire sandwich',
       'walking w/ something in mouth', 'burying nut', 'begging',
       'was hanging out w/ #2 until i showed up', 'quietly',
       'nest building', 'burying', 'dead (something ran over it)',
       'chased #12', 'playing (with #9)', 'sitting still',
       'sitting in tree', 'together w/ #7', 'not sure

#Which Features to Exclude

I would exclude the following features


*   Unique Squirrel ID (Because there are some duplicates and it could leak)
*   Target (It will definitley leak because it is what I'm looking for)
*   All 3 Columns used to make target (Definitley Leakage)
*   Lat/Long Point (Because it's a duplicate of X and Y Columns)
*   I'm still deciding what needs to be dropped








